In [1]:
import re
import time
import os

import pandas as pd
from tqdm import tqdm

In [2]:
banks = pd.read_csv("banks.csv", header=1, index_col=0).rename(
    columns={"Unnamed: 1": "name", "Unnamed: 3": "company", "Unnamed: 8": "orgn"}
)
banks.head()

,name,oldcregnr,company,cnamer,cdreg,lic,strcuraddr,orgn,2020,2021,2022,Unnamed: 12,student name,Комментарии
1,1481,NaN,ПАО Сбербанк,СБЕРБАНК РОССИИ,20.06.1991,NaN,"Российская Федерация, город Москва, улица Вави...",1027700132195,1080p-bps-sberbank-GO2020rjg2.pdf,sber_impact_report_for_2021_ru.pdf,sber-ar-2022-ru.pdf,https://www.sberbank.ru/,Sukharev A. F.,NaN
2,1000,NaN,Банк ВТБ (ПАО),ВТБ,17.10.1990,NaN,"191144, г. Санкт-Петербург, Дегтярный переулок...",1027739609391,VTB_Sustainability_Report_2020_rus.pdf,20221207-VTB_Sustainability_Report_2021_rus.pdf,VTB_Sustainability_Report_2022_rus.pdf,ВТБ,Sukharev A. F.,NaN
3,354,NaN,Банк ГПБ (АО),ГАЗПРОМБАНК,23.01.1992,NaN,"117420, г. Москва, ул. Наметкина, д. 16, корп...",1027700167110,gazprom-annual-report-2020-ru.pdf,gazprom-annual-report-2021-ru.pdf,gazprom-annual-report-2022-ru.pdf,Газпромбанк,Sukharev A. F.,NaN
4,1326,NaN,"АО ""АЛЬФА-БАНК""",АЛЬФА-БАНК,03.01.1991,NaN,"107078, г. Москва, ул. Каланчевская, д. 27",1027700067328,Альфа банк ГО.pdf,-,-,https://alfabank.ru/,Borodkina DV,NaN
5,3349,NaN,"АО ""Россельхозбанк""",РОССЕЛЬХОЗБАНК,24.04.2000,NaN,"119034, г. Москва, Гагаринский пер., д.3",1027700342890,-,-,-,https://www.rshb.ru/,Borodkina DV,Только финансовые отчёты


In [3]:
pdf_folder = "banks"
pdfs = os.listdir(pdf_folder)
len(pdfs)

90

In [11]:
files = []
for idx, row in banks.iterrows():
    file_2020 = row["2020"]
    file_2021 = row["2021"]
    file_2022 = row["2022"]
    if not file_2020 in pdfs:
        if pd.isna(file_2020) or file_2020 == "-" or file_2020 == " -":
            file_2020 = None
        else:
            print(f"2020: {file_2020} not found")

    if not file_2021 in pdfs:
        if pd.isna(file_2021) or file_2021 == "-" or file_2021 == " -":
            file_2021 = None
        else:
            print(f"2021: {file_2021} not found")

    if not file_2022 in pdfs:
        if pd.isna(file_2022) or file_2022 == "-" or file_2022 == " -":
            file_2022 = None
        else:
            print(f"2022: {file_2022} not found")

    files.extend(
        [
            {
                "company": row["company"],
                "name": row["cnamer"],
                "reg": row["name"],
                "orgn": row["orgn"],
                "file_path": os.path.join(pdf_folder, file_2020) if file_2020 else None,
                "year": 2020,
            },
            {
                "company": row["company"],
                "name": row["cnamer"],
                "reg": row["name"],
                "orgn": row["orgn"],
                "file_path": os.path.join(pdf_folder, file_2021) if file_2021 else None,
                "year": 2021,
            },
            {
                "company": row["company"],
                "name": row["cnamer"],
                "reg": row["name"],
                "orgn": row["orgn"],
                "file_path": os.path.join(pdf_folder, file_2022) if file_2022 else None,
                "year": 2022,
            },
        ]
    )

In [12]:
files = pd.DataFrame(files)

# Parse per paragraph

In [6]:
import os
import re
from collections import defaultdict

import textract
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
from PyPDF2 import PdfReader, PdfWriter
from tqdm.autonotebook import tqdm
import pandas as pd

/home/samoed/.pyenv/versions/3.10.4/lib/python3.10/site-packages/scipy/__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.26.0 is required for this version of SciPy (detected version 1.26.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/tmp/ipykernel_63412/945261966.py:10: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [7]:
REPORTS_DIRECTORY = "reports"

RE_DERIVED = re.compile(r"\w+( -|- |-|! - )\w+")
RE_RUSSIAN_TEXT = re.compile(r"[а-яА-Яa-zA-Z0-9\-]+")

stopwords_ru = stopwords.words("russian")

In [8]:
morph = MorphAnalyzer()
cp_names = {
    "ак",
    "акб",
    "акига",
    "алмазэргиэнбанк",
    "алрос",
    "альфабанк",
    "алюминиевый",
    "амвэя",
    "ангарский",
    "анк",
    "ао",
    "архангельский",
    "ас",
    "асэ",
    "атомпроект",
    "атомредметзолото",
    "атомэнергомаш",
    "атомэнергопроект",
    "атомэнергопром",
    "афк",
    "ашан",
    "аэрофлот",
    "балтика",
    "барс",
    "бат",
    "башнефть",
    "билайн",
    "бифид",
    "бф",
    "виктория",
    "владивостокский",
    "внипиэт",
    "волга",
    "волгателек",
    "волжский",
    "восток",
    "втб",
    "выксунский",
    "вымпелком",
    "вэб",
    "газпром",
    "газпромбанк",
    "гарантинвест",
    "генерировать",
    "геннадий",
    "гидропресс",
    "гк",
    "гмк",
    "гнц",
    "головной",
    "гольцблат",
    "грэс",
    "гтлк",
    "губкин",
    "дания",
    "дивизион",
    "евразийский",
    "еврохим",
    "елена",
    "енисейский",
    "еэс",
    "желдортранс",
    "зао",
    "зарубежнефть",
    "ик",
    "илим",
    "имииафрикантовый",
    "инжиниринговый",
    "инккапитал",
    "интер",
    "интернэшнл",
    "информация",
    "иркутск",
    "казаньоргсинтез",
    "казмунайгаз",
    "калининградский",
    "камаз",
    "касперский",
    "кб",
    "кемикалс",
    "ковровский",
    "комстаротс",
    "красцветмет",
    "крафтфудс",
    "кубань",
    "кузбассэнерго",
    "ленэнерго",
    "леруа",
    "лпк",
    "лср",
    "лукойл",
    "мвидео",
    "мдмбанк",
    "мегафон",
    "мерлен",
    "металлоинвест",
    "металлургический",
    "метафракс",
    "ммк",
    "монди",
    "мон’дэлиса",
    "моррис",
    "мосводоканал",
    "московский",
    "мрск",
    "мрф",
    "мсп",
    "мтс",
    "мхк",
    "неманский",
    "нестля",
    "ниаэп",
    "нижнекамскнефтехим",
    "нииар",
    "нипигаз",
    "ниу",
    "нк",
    "нлмк",
    "новатэк",
    "новогор",
    "новосибирский",
    "нордиск",
    "норильский",
    "оао",
    "объединение",
    "огк",
    "окб",
    "окбм",
    "около",
    "омк",
    "они",
    "ооо",
    "пао",
    "пейп",
    "пепеляев",
    "петропавловск",
    "пивоваренный",
    "пивоваров",
    "пигмент",
    "пик",
    "по",
    "полиметалл",
    "приволжье",
    "прикамье",
    "рао",
    "распадский",
    "ргу",
    "реновый",
    "ржд",
    "рксхолдинг",
    "рольф",
    "росатом",
    "росводоканал",
    "роснефть",
    "россеть",
    "российский",
    "россия",
    "ростелеком",
    "ростёха",
    "росэнергоатом",
    "рудна",
    "русало",
    "русгидро",
    "руссинвест",
    "руссия",
    "русый",
    "русь",
    "рф",
    "рязанский",
    "сабмиллер",
    "санофироссия",
    "сахалин",
    "сбербанк",
    "свез",
    "севергазпром",
    "севернефтегазпром",
    "северсталь",
    "сзлк",
    "сибирь",
    "сибурхолдинг",
    "система",
    "ситибанк",
    "ситроникс",
    "совкомбанк",
    "спбаэп",
    "суал",
    "сургутнефтегаз",
    "суэк",
    "схк",
    "сыктывкарский",
    "такед",
    "таманьнефтегаз",
    "татнефть",
    "твэл",
    "тгк",
    "техснабэкспорт",
    "тимченко",
    "тмк",
    "тнк",
    "тнквр",
    "тоаз",
    "трансаэро",
    "трансгаз",
    "транснефть",
    "тюмень",
    "урал",
    "уралкалий",
    "уралсиб",
    "уральский",
    "урсабанк",
    "ухта",
    "фгк",
    "ферреро",
    "фиабанк",
    "филиал",
    "филип",
    "фк",
    "фортум",
    "фосагро",
    "фпк",
    "фск",
    "химконцентрат",
    "центринвест",
    "черкизовый",
    "чусовской",
    "шелл",
    "шереметьево",
    "щуровский",
    "эксонмобила",
    "энела",
    "энергохолдинг",
    "энерджи",
    "эсэфай",
    "южнороссийский",
    "юкос",
    "юникредитбанк",
    "юнипро",
    "яндекс",
    "ятэк",
}

In [17]:
def parse_pages_to_texts(files: pd.DataFrame):
    errors = []
    df = []
    for idx, row in tqdm(files.iterrows(), total=files.shape[0]):
        report_name = row["file_path"]
        company_name = row["company"]
        company_reg = row["reg"]
        year = row["year"]

        if not report_name:
            continue

        if not report_name.endswith(".pdf"):
            print(report_name)
            continue
        try:
            report = textract.process(report_name)
        except Exception as e:
            errors.append((report_name, e))
            continue

        for i, paragraph in enumerate(report.decode().split("\n\n")):
            paragraph = paragraph.replace("\n", " ").lower()
            report_page = RE_DERIVED.sub("", paragraph)
            report_page_lst = word_tokenize(report_page)
            tokens = []
            for word_ in filter(RE_RUSSIAN_TEXT.match, report_page_lst):
                if word_ and word_ not in stopwords_ru and word_ not in cp_names:
                    word_ = word_.strip()
                    word_ = morph.parse(word_)[0]
                    normal_form = word_.normal_form
                    tokens.append(normal_form)

            df.append(
                {
                    "index": company_reg,
                    "company": company_name,
                    "year": year,
                    "paragraph": i,
                    "original_text": paragraph,
                    "cleaned_text": " ".join(tokens),
                }
            )
    print(*errors, sep="\n")
    return df

In [18]:
df = parse_pages_to_texts(files)

  0%|          | 0/300 [00:00<?, ?it/s]

In [19]:
new_df = pd.DataFrame(df)
new_df.head()

,index,company,year,paragraph,original_text,cleaned_text
0,1481,ПАО Сбербанк,2020,0,годовой отчет 2020,годовой отчёт 2020
1,1481,ПАО Сбербанк,2020,1,содержание,содержание
2,1481,ПАО Сбербанк,2020,2,содержание 3,содержание 3
3,1481,ПАО Сбербанк,2020,3,обращение председателя правления оао«бпс-сберб...,обращение председатель правление
4,1481,ПАО Сбербанк,2020,4,27,27


In [21]:
new_df.to_csv("paragraphs_banks.csv.zip")